### Auction Theory
> Auctions are bayesian games of incomplete information. The code below analysis auctions from a game theory perspective

![Auctions](https://qph.ec.quoracdn.net/main-qimg-ffdfa32fb01c75477ef67c7a3a05daa1-c)
Source: https://www.quora.com/What-are-the-best-books-to-start-learning-about-Auction-Theory

### About this Notebook
> The following is an experimental code to study different types of auctions. This is a part of CS6283 - Computational Social Choice. The code is written using Python 3.x and requires Ipython, numpy and itertools modules. 

> Author: _Sivam Pillai_

Running the code is simple. Just run each cell sequentially.
* Run the first cell, which generates an html file where you can enter the game specifications
* The following cells go through a number of computation separated into modules for easy flow and understanding
* The last cell shows optimal allocation and individual player's payments in a tabular form

***

<div class="alert alert-block alert-info">Tip: In this notebook the terms Players and Bidders are used interchangeably.</div>

In [329]:
from IPython.display import HTML

input_form = """
<div style="background-color:gainsboro; border:solid black;">
<div><b>Enter the number of items and bidders for the Auction:</b></div>
<div style="display:inline">
Players: <input type="text" id="var_players" value="1">
Items  : <input type="text" id="var_items" value="2">
</div>
<button onclick="gen_table()">Generate Table</button>
</div>
<style>
    table{
        width:100%;
        height:10)%;
    }
    table td{
        padding:5px;
        margin:5px;
        border:1px solid #ccc;
    }
</style>
<div id="box">
<table id="basicTable">
</table>
</div>
<button onclick="set_valuations()">Set Values</button>
"""



javascript = """
<script type="text/Javascript">
    function gen_table(){
        var players = document.getElementById('var_players').value;
        var items = document.getElementById('var_items').value;
        
        var parent = document.getElementById('box');
        var child = document.getElementById('basicTable');
        parent.removeChild(child);
        mytable = $('<table></table>').attr({ id: "basicTable" });
        var rows = players;
        var cols = items;
        var tr = [];
        var row = $('<tr><td></td></tr>').attr({ class: ["class1"].join(' ') }).appendTo(mytable);
        for (var j = 0; j < cols; j++) {
                $("<td>Item "+j+"</td>").appendTo(row); 
            }
        for (var i = 0; i < rows; i++) {
            var row = $('<tr></tr>').attr({ class: ["class1"].join(' ') }).appendTo(mytable);
            $("<td>Player "+i+"</td>").appendTo(row)
            for (var j = 0; j < cols; j++) {
                $("<td><input type='text' id='val_"+i+"_"+j+"' value='1'></td>").appendTo(row); 
            }

        }
        console.log("TTTTT:"+mytable.html());
        mytable.appendTo("#box"); 
    }

    function set_valuations(){
        var players = document.getElementById('var_players').value;
        var items = document.getElementById('var_items').value;
        var val = [];
        var x;
        for (var i = 0; i < players; i++) {
            val[i] = [];
            for (var j = 0; j < items; j++) {
                x = 'val_'+i+'_'+j;
                val[i][j] = document.getElementById(x).value;
            }
        }
        
        var kernel = IPython.notebook.kernel;
        var cmd1 = "players = " + players;
        var cmd2 = "items = " + items;
        var cmd3 = "my_val = " + val;
        
        console.log("Executing Command: " + cmd1);     
        kernel.execute(cmd1);
        
        console.log("Executing Command: " + cmd2);
        kernel.execute(cmd2);  
        
        console.log("Executing Command: " + cmd3);
        kernel.execute(cmd3); 

    }
</script>
"""

HTML(input_form + javascript)

In [340]:
# First compute all different one-to-one allocations of items to bidders. 
import numpy as np
import itertools

k = 0
allocs = np.zeros((items, players, 2),dtype=int)
valuations = np.zeros((players, items),dtype=int)
for i in range(0,players):
    for j in range(0,items):
        allocs[j,i] = [j,i]
        valuations[i,j] = my_val[k]
        k += 1

In [345]:
# Identify all possible allocations of items to bidders
########
# Uncomment the following lines to use the default values for items, players and valuations
# items = 4
# players = 3
# allocs = np.array([[[0,0],[0,1],[0,2]],
#                    [[1,0],[1,1],[1,2]],
#                    [[2,0],[2,1],[2,2]],
#                    [[3,0],[3,1],[3,2]],])
# valuations = np.array([[10,20,30,40],
#                        [20,20,20,40],
#                        [40,10,10,40]])
########

x = list(itertools.product(*allocs))
r = len(x)
c = len(x[0])

p = np.zeros((r,items+players+1),dtype=int)

for i in range(0,r):
    for j in range(0,c):
        plid = x[i][j][1]+players # initial columns are items
        if x[i][j][1] == 0:
            p[i,plid] = p[i,plid] + valuations[x[i][j][1], x[i][j][0]]
            p[i,x[i][j][0]] = 0
        if x[i][j][1] == 1:
            p[i,plid] = p[i,plid] + valuations[x[i][j][1], x[i][j][0]]
            p[i,x[i][j][0]] = 1
        if x[i][j][1] == 2:
            p[i,plid] = p[i,plid] + valuations[x[i][j][1], x[i][j][0]]
            p[i,x[i][j][0]] = 2
    p[i,-1] = np.sum(p[i,-4:-1])

In [333]:
# Store the data in the form of a Dataframe
import pandas as pd

auctions_df = pd.DataFrame(p)
auctions_df.columns = ['i'+ str(i) for i in range(0,items)] + \
                      ['v'+ str(i) for i in range(0,players)] + \
                      ['v_tot']
#auctions_df.columns = ['i1','i2','i3','i4','v1','v2','v3','v_tot']
v_max_without = np.zeros(players,dtype=int)
for i in range(0,players):
    v_max_without[i] = max(auctions_df.v_tot[(auctions_df.filter(regex='i') != 0).all(1)])

### Vickrey–Clarke–Groves Auction (VCG)

In [335]:
vcg = auctions_df[auctions_df.v_tot == max(auctions_df.v_tot)].copy().reset_index(drop=True)
for i in range(0,players):
    vcg.loc[:,'p'+str(i)] = v_max_not[i] - (vcg.v_tot - vcg['v'+str(i)])

In [336]:
col_names = list(vcg)
print('Items: ',[s for s in col_names if 'i' in s[0]])
print('Valuation of Player to their allocation: ',[s for s in col_names if 'v' in s[0] and '_' not in s])
print('Payment of Player for their allocation: ',[s for s in col_names if 'p' in s[0]])
HTML(vcg.to_html())

Items:  ['i0', 'i1']
Valuation of Player to their allocation:  ['v0', 'v1']
Payment of Player for their allocation:  ['p0', 'p1']


,i0,i1,v0,v1,v_tot,p0,p1
0,0,0,90,0,90,120,40
